In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

## Options

In [61]:
# parse options
problem = 'twelve_pieces_process.json'
problem_subdir = 'results'

recompute_action_states = False
load_external_movements = False

In [62]:
# generic planning options
id_only = None # 'A2_M1' # None | 'A273_M0'
free_motion_only = False
low_res = True

viz_upon_found = False
propagate_only = False
plan_impacted = False

In [78]:
# client options
viewer = True
debug = True
verbose = True
diagnosis = False

watch = True
step_sim = True

disable_env = False
reinit_tool = False

write = False
save_now = False

In [79]:
from collections import namedtuple
PlanningArguments = namedtuple('PlanningArguments', ['problem', 'debug', 'diagnosis', 'id_only', 'free_motion_only', 'viz_upon_found', 
                                             'save_now', 'propagate_only', 'write', 'plan_impacted', 'watch', 'step_sim', 'verbose'])

args = PlanningArguments(problem, debug, diagnosis, id_only, free_motion_only, viz_upon_found, save_now, propagate_only, write, plan_impacted, watch, step_sim, verbose)

## Parse process from json

In [5]:
import os
from termcolor import cprint
import pybullet_planning as pp
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, get_process_path, save_process

In [6]:
process = parse_process(problem, subdir=problem_subdir)
result_path = get_process_path(problem, subdir='results')
if len(process.movements) == 0:
    cprint('No movements found in process, trigger recompute actions.', 'red')
    recompute_action_states = True
if recompute_action_states:
    cprint('Recomputing Actions and States', 'cyan')
    recompute_action_states(process)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210128_RemodelFredPavilion\results\twelve_pieces_process.json


In [7]:
# # force load external if only planning for the free motions
# load_external_movements = load_external_movements or free_motion_only or id_only is not None
# if load_external_movements:
#     ext_movement_path = os.path.dirname(result_path)
#     cprint('Loading external movements from {}'.format(ext_movement_path), 'cyan')
#     process.load_external_movements(ext_movement_path)
#     if recompute_action_states:
#         save_process(process, result_path)
#         cprint('Recomputed process saved to %s' % result_path, 'green')

Loading external movements from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210128_RemodelFredPavilion\results


## Start client

In [68]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
client, robot, _ = load_RFL_world(viewer=viewer or diagnosis or view_states or watch or step_sim, verbose=False)
set_initial_state(client, robot, process, disable_env=disable_env, reinit_tool=reinit_tool)

In [70]:
from integral_timber_joints.planning.robot_setup import GANTRY_ARM_GROUP, GANTRY_GROUP, BARE_ARM_GROUP
from compas.robots import Joint
import numpy as np

joint_names = robot.get_configurable_joint_names(group=GANTRY_ARM_GROUP)
joint_types = robot.get_joint_types_by_names(joint_names)
# 0.1 rad = 5.7 deg
joint_jump_threshold = {jt_name : np.pi/6 \
        if jt_type in [Joint.REVOLUTE, Joint.CONTINUOUS] else 0.1 \
        for jt_name, jt_type in zip(joint_names, joint_types)}

options = {
    'debug' : debug,
    'diagnosis' : False,
    'low_res' : low_res,
    'distance_threshold' : 0.0012,
    'frame_jump_tolerance' : 0.0012,
    'verbose' : verbose,
    'jump_threshold' : joint_jump_threshold,
#     'max_distance' : args.max_distance,
}

# Plan only one movement

In [29]:
# if id_only:
#     beam_id = process.get_beam_id_from_movement_id(id_only)
#     process.get_movement_summary_by_beam_id(beam_id)

In [39]:
from integral_timber_joints.planning.stream import compute_free_movement, compute_linear_movement
from integral_timber_joints.planning.solve import compute_movement

chosen_m = process.get_movement_by_movement_id(id_only)
compute_movement(client, robot, process, chosen_m, options=lm_options, diagnosis=diagnosis)

RoboticLinearMovement(#A2_M1, Linear Advance to Final Frame of Beam ('b0'), traj 1)
end conf FK inconsistent (0.00005 m) with given current frame in end state.
Both start/end confs are pre-specified, problem might be too stiff to be solved.
One-sided Cartesian planning : start conf set, forward mode
	cartesian trial #0
Plan found by IterativeIK! After 0 path failure (by IterativeIK) over 1 samples.


True

In [40]:
from integral_timber_joints.planning.visualization import visualize_movement_trajectory

with pp.WorldSaver():
    visualize_movement_trajectory(client, robot, process, chosen_m, step_sim=True)

===
Viz:
RoboticLinearMovement(#A2_M1, Linear Advance to Final Frame of Beam ('b0'), traj 1)


Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
End state. 


# Plan movements for a beam

In [76]:
from integral_timber_joints.planning.run import compute_movements_for_beam_id

beam_id = 'b0'
# beam_id = process.get_beam_id_from_movement_id(args.id_only)
compute_movements_for_beam_id(client, robot, process, beam_id, args, opti
                              ons=options)




* compute ['RoboticLinearMovement', 'RoboticClampSyncLinearMovement'] (priority 1, status [<MovementStatus.neither_done: 4>, <MovementStatus.one_sided: 2>])



* compute ['RoboticLinearMovement'] (priority 0, status [<MovementStatus.one_sided: 2>])



* compute ['RoboticLinearMovement'] (priority 0, status [<MovementStatus.neither_done: 4>, <MovementStatus.one_sided: 2>])



* compute ['RoboticFreeMovement'] (priority 0, status [<MovementStatus.both_done: 3>, <MovementStatus.one_sided: 2>])



Visualize results
===
Viz:
RoboticFreeMovement(#A0_M0, Free Move reach Storage Approach Frame of PG1000 ('g2'), to get tool., traj 1)


Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
End state. 


===
Viz:
RoboticLinearMovement(#A0_M1, Linear Advance to Storage Frame of PG1000 ('g2'), to get tool., traj 1)


Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
End state. 


===
Viz:


...

===
Viz:


End state. 


===
Viz:
RoboticLinearMovement(#A3_M4, Linear Retract from storage after placing PG1000 ('g2') in storage, traj 1)


Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
End state. 


AttributeError: 'PlanningArguments' object has no attribute 'verbose'

# Disconnect client

In [77]:
client.disconnect()